In [11]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from matplotlib.pyplot import imshow
%matplotlib inline
from keras.layers.merge import concatenate
from keras.regularizers import l2
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD
import sys
import keras.backend as K
K.set_learning_phase(1)

In [12]:
def bn_inception(X_input, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
    conv_1x1 = Conv2D(filters=filters_1x1, kernel_size=(1, 1), padding='same')(X_input)
    conv_1x1 = BatchNormalization(axis = 3)(conv_1x1)
    conv_1x1 = Activation('relu')(conv_1x1)

    conv_3x3_reduce = Conv2D(filters=filters_3x3_reduce, kernel_size=(1, 1), padding='same')(X_input)
    conv_3x3_reduce = BatchNormalization(axis = 3)(conv_3x3_reduce)
    conv_3x3_reduce = Activation('relu')(conv_3x3_reduce)

    conv_3x3 = Conv2D(filters=filters_3x3, kernel_size=(3, 3), padding='same')(conv_3x3_reduce)
    conv_3x3 = BatchNormalization(axis = 3)(conv_3x3)
    conv_3x3 = Activation('relu')(conv_3x3)

    conv_5x5_reduce  = Conv2D(filters=filters_5x5_reduce, kernel_size=(1, 1), padding='same', activation='relu')(X_input)
    conv_5x5_reduce = BatchNormalization(axis = 3)(conv_5x5_reduce)
    conv_5x5_reduce = Activation('relu')(conv_5x5_reduce)

    conv_5x5 = Conv2D(filters=filters_5x5, kernel_size=(5, 5), padding='same', activation='relu')(conv_5x5_reduce)
    conv_5x5 = BatchNormalization(axis = 3)(conv_5x5)
    conv_5x5 = Activation('relu')(conv_5x5)

    maxpool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(X_input)

    maxpool_proj = Conv2D(filters=filters_pool_proj, kernel_size=(1, 1), strides=(1, 1), padding='valid', activation='relu')(maxpool)
    maxpool_proj = BatchNormalization(axis = 3)(maxpool_proj)
    maxpool_proj = Activation('relu')(maxpool_proj)

    inception_output = concatenate([conv_1x1, conv_3x3, conv_5x5, maxpool_proj], axis=3)  

    return inception_output

In [13]:
def Inception_v2(input_shape):
    X_input = Input(shape=input_shape)

    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')(X_input)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(X)
    
    X = BatchNormalization(axis=3)(X)

    X = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=192, kernel_size=(3, 3), padding='same', activation='relu')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = BatchNormalization(axis=3)(X)

    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(X)

    X = bn_inception(X, 64, 96, 128, 16, 32, 32)

    X = bn_inception(X, 128, 128, 192, 32, 96, 64)

    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(X)

    X = bn_inception(X, 192, 96, 208, 16, 48, 64)
    
    X = bn_inception(X, 160, 112, 224, 24, 64, 64)

    X = bn_inception(X, 128, 128, 256, 24, 64, 64)

    X = bn_inception(X, 112, 144, 288, 32, 64, 64)
    
    X = bn_inception(X, 256, 160, 320, 32, 128, 128)

    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(X)

    X = bn_inception(X, 256, 160, 320, 32, 128, 128)

    X = bn_inception(X, 384, 192, 384, 48, 128, 128)

    X = AveragePooling2D(pool_size=(7, 7))(X)
    
    X = Dropout(0.4)(X)

    X = Dense(1024, activation='relu')(X)
    
    X = Dense(2, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X, name='Inception_v2')
    
    return model


In [14]:
model = Inception_v2((224,224,3))

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_115 (Conv2D)             (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_119 (BatchN (None, 112, 112, 64) 256         conv2d_115[0][0]                 
__________________________________________________________________________________________________
activation_115 (Activation)     (None, 112, 112, 64) 0           batch_normalization_119[0][0]    
__________________________________________________________________________________________________
max_poolin

In [16]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,epochs = 900, batch_size=32,validation_data=(x_test,y_test), shuffle = True)